In [1]:
import numpy as np
import pandas as pd
import joblib

In [2]:
data = pd.read_csv('bez_nedostajucih.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [4]:
def mapi(str):
    if 'Summer' in str:
        return 0
    if 'Winter' in str:
        return 1
    else:
        return None

In [5]:
nesto = data.drop(columns = ['ID', 'Name', 'Team', 'City', 'Event', 'Season', 'Year'])

mapiranje = {'M' : 0, 'F' : 1}

nesto['Sex'] = nesto['Sex'].map(mapiranje)
nesto['Sport'] = label_encoder.fit_transform(nesto['Sport'])
nesto['NOC'] = label_encoder.fit_transform(nesto['NOC'])
nesto['Games'] = nesto['Games'].map(mapi)

In [6]:
y = data['Medal']
x = nesto.columns.to_list()[:-1]

In [7]:
X = nesto[x]
Y = y.fillna('No')

## Balansiranje podataka

 Zato sto imamo mnogo vise ljudi bez medalje (45000) nego ljudi sa medaljom (3*2000)

In [8]:
def resample(sampler, X, Y):
    X_resampled, y_resampled = sampler.fit_resample(X, Y)
    print(colored("Number of instances before resampling with {} : {}. ".format(type(sampler).__name__,\
                                                                           Counter(Y).items()), "green"))
    print(colored("Number of instances after  resampling with {} : {}. ".format(type(sampler).__name__,\
                                                                           Counter(y_resampled).items()), "blue"))
    return X_resampled, y_resampled

In [9]:
from imblearn.combine import SMOTEENN
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter
from termcolor import colored

In [10]:
# sampler = SMOTE(sampling_strategy='auto', random_state=42)

In [11]:
# X_resampled, Y_resampled = resample(sampler, X, Y)

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler

In [22]:
smoteenn = SMOTEENN(random_state=22, sampling_strategy='auto')
X_resampled, y_resampled = resample(smoteenn, X, Y)

Number of instances before resampling with SMOTEENN : dict_items([('No', 45165), ('Bronze', 2346), ('Gold', 2429), ('Silver', 2326)]). 
Number of instances after  resampling with SMOTEENN : dict_items([('Bronze', 36299), ('Gold', 36438), ('No', 26405), ('Silver', 35992)]). 


In [24]:
X_resampled.size, y_resampled.size

(945938, 135134)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=22, stratify=y_resampled)

In [27]:
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((94593, 7), (40541, 7), (94593,), (40541,))

In [28]:
medalje = y_resampled.value_counts()
print(medalje)

Medal
Gold      36438
Bronze    36299
Silver    35992
No        26405
Name: count, dtype: int64


In [29]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train = pd.DataFrame(data = X_train_scaled, columns = X_train.columns)
X_test = pd.DataFrame(data = X_test_scaled, columns = X_test.columns)

In [30]:
joblib.dump((X_train, X_test, y_train, y_test), 'classification_data.joblib')

['classification_data.joblib']